In [2]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
base_dir = '/home/ankit/kaggle_comp/kaggle/chest_xray/'

In [4]:
train_dir = os.path.join(base_dir, 'train')

In [5]:
# Directory with our training images
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')

In [6]:
train_normal_fnames = os.listdir(train_normal_dir)
train_pneumonia_fnames = os.listdir(train_pneumonia_dir)

In [7]:
val_dir = os.path.join(base_dir, 'val')

In [8]:
# Directory with our training images
val_normal_dir = os.path.join(val_dir, 'NORMAL')
val_pneumonia_dir = os.path.join(val_dir, 'PNEUMONIA')

In [9]:
val_normal_fnames = os.listdir(val_normal_dir)
val_pneumonia_fnames = os.listdir(val_pneumonia_dir)

In [10]:
test_dir = os.path.join(base_dir, 'test')

In [11]:
# Directory with our training images
test_normal_dir = os.path.join(test_dir, 'NORMAL')
test_pneumonia_dir = os.path.join(test_dir, 'PNEUMONIA')

In [12]:
test_normal_fnames = os.listdir(test_normal_dir)
test_pneumonia_fnames = os.listdir(test_pneumonia_dir)

In [13]:
print("There are total ", len(train_normal_fnames) - 1, "Normal images in the 'Train' dataset.")
print("There are total ", len(train_pneumonia_fnames) - 1, "Pneumonia images in the 'Train' dataset.")
print("There are total ", len(val_normal_fnames) -1, "Normal images in the 'Validation' dataset.")
print("There are total ", len(val_pneumonia_fnames) -1, "Penumonia images in the 'Validation' dataset.")
print("There are total ", len(test_normal_fnames) -1, "Normal images in the 'Test' dataset.")
print("There are total ", len(test_pneumonia_fnames) -1, "Penumonia images in the 'Test' dataset.")

There are total  1341 Normal images in the 'Train' dataset.
There are total  3875 Pneumonia images in the 'Train' dataset.
There are total  8 Normal images in the 'Validation' dataset.
There are total  8 Penumonia images in the 'Validation' dataset.
There are total  233 Normal images in the 'Test' dataset.
There are total  389 Penumonia images in the 'Test' dataset.


In [14]:
train_datagen = ImageDataGenerator(rescale=1.0/255.)
val_datagen = ImageDataGenerator(rescale=1.0/255.)
test_datagen = ImageDataGenerator(rescale=1.0/255.)

In [15]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [26]:
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [27]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=50,
                                                    color_mode= 'rgb',
                                                    class_mode='binary',
                                                    target_size=(300, 300))   

Found 5216 images belonging to 2 classes.


In [28]:
val_generator =  val_datagen.flow_from_directory(val_dir,
                                                 batch_size=50,
                                                 color_mode= 'rgb',
                                                 class_mode='binary',
                                                 target_size = (300, 300))

Found 16 images belonging to 2 classes.


In [29]:
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                 batch_size=50,
                                                 color_mode= 'rgb',
                                                 class_mode='binary',
                                                 target_size = (300, 300))

Found 624 images belonging to 2 classes.


In [30]:
history = model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=50,
                    epochs=10,
                    validation_steps=None,
                    verbose=1)

Epoch 1/10
50/50 [==============================] - 311s 6s/step - loss: 0.5894 - accuracy: 0.7392 - val_loss: 0.5708 - val_accuracy: 0.6731
Epoch 2/10
50/50 [==============================] - 301s 6s/step - loss: 0.3310 - accuracy: 0.8710 - val_loss: 0.3588 - val_accuracy: 0.8526
Epoch 3/10
50/50 [==============================] - 296s 6s/step - loss: 0.2311 - accuracy: 0.9112 - val_loss: 0.4349 - val_accuracy: 0.8029
Epoch 4/10
50/50 [==============================] - 300s 6s/step - loss: 0.1797 - accuracy: 0.9328 - val_loss: 0.5157 - val_accuracy: 0.7692
Epoch 5/10
50/50 [==============================] - 298s 6s/step - loss: 0.1501 - accuracy: 0.9460 - val_loss: 0.7391 - val_accuracy: 0.7003
Epoch 6/10
50/50 [==============================] - 301s 6s/step - loss: 0.1533 - accuracy: 0.9408 - val_loss: 0.5529 - val_accuracy: 0.7804
Epoch 7/10
50/50 [==============================] - 296s 6s/step - loss: 0.1255 - accuracy: 0.9512 - val_loss: 1.1068 - val_accuracy: 0.6715
Epoch 8/10
50

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
img = mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg')

In [ ]:
img.shape

In [ ]:
plt.imshow(mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg'))

In [ ]:
for i in train_normal_fnames:
    img = mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/NORMAL/' + tr)
    size = img.shape
    print(size)

In [ ]:
plt.imshow(mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg'))

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_normal_pix = [os.path.join(train_normal_dir, fname) 
                for fname in train_normal_fnames[ pic_index-8:pic_index]]

next_pneumonia_pix = [os.path.join(train_pneumonia_dir, fname) 
                for fname in train_pneumonia_fnames[ pic_index-8:pic_index]]

In [ ]:
for i, img_path in enumerate(next_normal_pix+next_penumonia_pix):

In [ ]:
for i, img_path in enumerate(next_normal_pix+next_penumonia_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()